In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, f1_score, make_scorer, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import xgboost
from xgboost import XGBClassifier
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE, ADASYN
import random
from random import randint
from sklearn.datasets import *
from IPython.display import Image, display_svg, SVG
import os
from sklearn.pipeline import Pipeline
from dtreeviz.trees import *
from sklearn.tree import plot_tree
os.environ["PATH"] += os.pathsep + "C:\\Users\\tmcro\\anaconda3\\pkgs\\graphviz-2.38-hfd603c8_2\\Library\\bin\\graphviz\\"
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.pipeline import Pipeline as imbpipeline # https://towardsdatascience.com/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion


In [2]:
import pandas as pd
df = pd.read_csv('heart_2020_cleaned.csv')

target_name = "HeartDisease"
y = df[target_name]
X = df.drop(columns=[target_name])

In [3]:
# Scoring Metric
def my_custom_score(y_true, y_pred):
    cf = confusion_matrix(y_true, y_pred)
    precision = cf[1,1] / sum(cf[:,1])
    recall    = cf[1,1] / sum(cf[1,:])
    f1_score  = 2*precision*recall / (precision + recall)
    rwf_score = 2*precision* (recall*2) /(precision + (recall*2))
    return rwf_score

my_scorer = make_scorer(my_custom_score, greater_is_better= True)

# Change class metric
class_metric = my_scorer

In [4]:
X.dtypes

BMI                 float64
Smoking              object
AlcoholDrinking      object
Stroke               object
PhysicalHealth      float64
MentalHealth        float64
DiffWalking          object
Sex                  object
AgeCategory          object
Race                 object
Diabetic             object
PhysicalActivity     object
GenHealth            object
SleepTime           float64
Asthma               object
KidneyDisease        object
SkinCancer           object
dtype: object

In [5]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [6]:
categorical_columns

['Smoking',
 'AlcoholDrinking',
 'Stroke',
 'DiffWalking',
 'Sex',
 'AgeCategory',
 'Race',
 'Diabetic',
 'PhysicalActivity',
 'GenHealth',
 'Asthma',
 'KidneyDisease',
 'SkinCancer']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
catcols = [1,2,3,6,7,8,9,10,11,12,14,15,16]

### SMOTENC

In [9]:
Smote_NC = SMOTENC(categorical_features= catcols, random_state=42)

In [10]:
print(y_train.value_counts())
# Fit SMOTE to training data
X_train_resampled, y_train_resampled = Smote_NC.fit_resample(X_train, y_train)
# Preview synthetic sample class distribution
print('\n')
print(pd.Series(y_train_resampled).value_counts())

No     219418
Yes     20428
Name: HeartDisease, dtype: int64


Yes    219418
No     219418
Name: HeartDisease, dtype: int64


In [11]:
X_train = X_train_resampled
y_train = y_train_resampled

In [12]:
# Identify Processors
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer([('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

smoter = SMOTENC(categorical_features= categorical_columns, random_state= 0)
classifier = LogisticRegression(max_iter=500)

### Code Additions

In [13]:
# SOURCE: The origin of this confusion matrix code was found on medium, '
# from https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):

    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            a = cf[0,0]
            b = cf[0,1]
            c = cf[1,0]
            d = cf[1,1]
            tn = ((a / (a+b))*100).round(2).astype(str) + '%'
            fp = ((b / (a+b))*100).round(2).astype(str) + '%'
            fn = ((c / (c+d))*100).round(2).astype(str) + '%'
            tp = ((d / (c+d))*100).round(2).astype(str) + '%'
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            rwf_score = 2*precision* (recall*2) /(precision + (recall*2))
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}\n\nRecall-Weighted F Score={:0.3f}".format(
                accuracy,precision,recall,f1_score, rwf_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""

    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages =  [tn,fp,fn,tp]
        # old = group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label', weight = 'bold')
        plt.xlabel('Predicted label' + stats_text, weight = 'bold')
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title,size = 20, weight = 'bold')

In [14]:
dfcols = ['Model', 'RWF Score', 'F1',  'Recall',  'Precision',  'Accuracy']
model_summary = pd.DataFrame(columns=dfcols)
model_summary

,Model,RWF Score,F1,Recall,Precision,Accuracy


In [15]:
# Define Result Saving Initial Function
def save_result(cf, model_name):
            global model_summary
            accuracy  = np.trace(cf) / float(np.sum(cf))
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            rwf_score = 2*precision* (recall*2) /(precision + (recall*2))
            row = [(model_name, rwf_score, f1_score, recall, precision, accuracy)]
            res = pd.DataFrame(columns = dfcols, data = row)
            yeep = [model_summary, res]
            model_summary = pd.concat(yeep)
            model_summary = model_summary.sort_values('RWF Score', ascending = False)
            model_summary = model_summary.drop_duplicates()
            return model_summary.round(3)

### Models to Run 

In [16]:
def run_model(model, model_name):
    model.fit(X_train, y_train)
    model_prediction = model.predict(X_test)
    cf_matrix = confusion_matrix(y_test, model_prediction)
    save_result(cf_matrix, model_name)
    return model_summary

In [17]:
# Initial Models
log_reg = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
decisionTree = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=5, random_state=0))
random_forest = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=50, random_state=0))
baggedTrees = make_pipeline(preprocessor, BaggingClassifier(DecisionTreeClassifier(), n_estimators=50, random_state=0))
extraTrees = make_pipeline(preprocessor, ExtraTreesClassifier(n_estimators=50, random_state=0))
knn = make_pipeline(preprocessor, KNeighborsClassifier(n_neighbors=5))
xgBoost = make_pipeline(preprocessor, XGBClassifier(n_estimators=50, random_state=0))


In [18]:
init_models = [log_reg, decisionTree, random_forest, baggedTrees, extraTrees, knn]

In [19]:
run_model(log_reg, 'Logistic Regression')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326


In [20]:
run_model(decisionTree, 'Decision Tree')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [21]:
run_model(random_forest, 'Random Forest')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [22]:
run_model(baggedTrees, 'Bagged Trees')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [23]:
run_model(extraTrees, 'Extra Trees')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,Extra Trees,0.346773,0.273155,0.321670,0.237357,0.851293
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [24]:
run_model(knn, 'KNN')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,KNN,0.351840,0.303777,0.555940,0.208985,0.778634
0,Extra Trees,0.346773,0.273155,0.321670,0.237357,0.851293
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [42]:
run_model(xgBoost, 'XGBoost')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,XGBoost,0.440315,0.343849,0.392369,0.306008,0.869917
0,XGBoost with Grid Search,0.418531,0.303705,0.276746,0.336485,0.889767
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Logistic Regression with Grid Search,0.380903,0.337541,0.741253,0.218524,0.747251
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,KNN,0.351840,0.303777,0.555940,0.208985,0.778634
0,Extra Trees,0.346773,0.273155,0.321670,0.237357,0.851293
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [43]:
xgBoost.get_params().values()

dict_values([None, [('columntransformer', ColumnTransformer(transformers=[('one-hot-encoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Smoking', 'AlcoholDrinking', 'Stroke',
                                  'DiffWalking', 'Sex', 'AgeCategory', 'Race',
                                  'Diabetic', 'PhysicalActivity', 'GenHealth',
                                  'Asthma', 'KidneyDisease', 'SkinCancer']),
                                ('standard_scaler', StandardScaler(),
                                 ['BMI', 'PhysicalHealth', 'MentalHealth',
                                  'SleepTime'])])), ('xgbclassifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weig

## Iterate the best

Lets take logistic regression and random forest. 

In [27]:
# Define Grid
rf_grid = [{'logisticregression__max_iter': [100, 500, 300], 
        'logisticregression__C': [1, 2,3] }]

In [28]:
logreg_gridsearch = GridSearchCV(estimator=log_reg, param_grid=rf_grid, scoring=class_metric, cv=5)

In [29]:
logreg_gridsearch.fit(X_train, y_train)
run_model(logreg_gridsearch, 'Logistic Regression with Grid Search')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,XGBoost,0.440315,0.343849,0.392369,0.306008,0.869917
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Logistic Regression with Grid Search,0.380903,0.337541,0.741253,0.218524,0.747251
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,KNN,0.351840,0.303777,0.555940,0.208985,0.778634
0,Extra Trees,0.346773,0.273155,0.321670,0.237357,0.851293
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251


In [44]:
xg_grid = [{'xgbclassifier__n_estimators': [50, 100],
        'xgbclassifier__max_depth': [5, 6, 7],
        'xgbclassifier__learning_rate': [0.2, 0.3, 0.4]}]

In [45]:
xgBoost_grid = GridSearchCV(estimator=xgBoost, param_grid=xg_grid, scoring=class_metric, cv=3)

In [46]:
xgBoost.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'xgbclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__one-hot-encoder', 'columntransformer__standard_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__standard_scaler__copy', 'columntransformer__standard_scaler__with_mean', 'columntransformer__standard_scaler__with_std', 'xgbclassifier__objective', 'xgbclassifier__base_score', 'xgbclassifier__booster', 'xgbclassifier__colsample_bylevel', 'xgbclassifier__colsample_bynode', 'xgbclassifier__colsample_bytree', 'xgbclassifier__gamma', 'xgbclassifier__gpu_id', 'xgbclassifier__impo

In [47]:
xgBoost_grid.fit(X_train, y_train)
run_model(xgBoost_grid, 'XGBoost with Grid Search')

,Model,RWF Score,F1,Recall,Precision,Accuracy
0,XGBoost,0.440315,0.343849,0.392369,0.306008,0.869917
0,XGBoost with Grid Search,0.418531,0.303705,0.276746,0.336485,0.889767
0,XGBoost with Grid Search,0.418110,0.305691,0.284233,0.330653,0.887841
0,Logistic Regression,0.381079,0.337694,0.741541,0.218628,0.747326
0,Logistic Regression with Grid Search,0.380903,0.337541,0.741253,0.218524,0.747251
0,Random Forest,0.373655,0.288472,0.316343,0.265114,0.864439
0,KNN,0.351840,0.303777,0.555940,0.208985,0.778634
0,Extra Trees,0.346773,0.273155,0.321670,0.237357,0.851293
0,Bagged Trees,0.334905,0.252925,0.258315,0.247756,0.867440
0,Decision Tree,0.305886,0.276928,0.731317,0.170803,0.668251
